## Aprendizagem Automática - Projeto

In [175]:
# Imports
import pandas as pd

# Models and selection methods
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import LinearSVR
# Linear regression metrics
from sklearn.metrics import explained_variance_score, mean_squared_error, max_error, mean_absolute_error
from scipy.stats import pearsonr
from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import LabelBinarizer

In [176]:
# Funcoes auxiliares

def printRegStatistics(truth, preds):
    print("The RVE is: ", explained_variance_score(truth, preds))
    print("The rmse is: ", mean_squared_error(truth, preds, squared=False))
    corr, pval = pearsonr(truth, preds)
    print("The Correlation Score is is: %6.4f (p-value=%e)\n"%(corr,pval))
    print("The Maximum Error is is: ", max_error(truth, preds))
    print("The Mean Absolute Error is: ", mean_absolute_error(truth, preds))
    

#Devolve uma lista com quantos elementos há seguidos entre valores de NaN. p.e: [1,nan,2,3,nan,nan,2,nan,2,3,4] -> [1,2,1,3]
def count_not_nan(serie):
    conta=0
    lista_seguida=[]
    for i in range(len(serie)):
        if serie[i]==1:
            conta+=1
        else:
            if conta!=0:
                lista_seguida.append(conta)
                conta=0
            else:
                conta=0
                
        if i==len(serie)-1 and conta!=0:
            lista_seguida.append(conta)
    return lista_seguida

#Função que lida com os nan nas df. Se o país não tiver nenhum dado -> drop. Se o país não tiver pelo menos lag+1 dados seguidos
#-> drop. O resto fica

def drop_nan(df,lag):
    pais_nan=[]
    for pais in df["Country Name"].index:
        nan_val=list(pd.DataFrame(df.iloc[pais][4:]).count(axis=1)) #ver se os valores são nan
        if sum(nan_val)==0: #se for tudo nan
            pais_nan.append(pais)
        elif max(count_not_nan(nan_val))<=lag: #Se não houver mais do que lag valores seguidos, drop
            pais_nan.append(pais)

    df=df.drop(pais_nan, axis=0)
    df = df.reset_index(drop=True)
    return df

#Função que escolha o dataframe até um certo ano apenas, que remove os países que contenham nan nesse intervalo de tempo

def indexing(dataf, ano_i):
    X_list=dataf.drop(columns=dataf.iloc[:,[1,2,3]]+dataf.loc[:,[str(i) for i in range(1960,ano_i)]])
    X_list=X_list.T
    X_list=X_list.rename(columns=X_list.iloc[0]).drop(X_list.index[0])
    return X_list

#Cria um DataFrame com os dados de um país e respetivos lags

def lags(df, lag, país):
    new_df=pd.DataFrame()
    new_df["dados"]=df[país]
    colunas = [str(i) + "lag" for i in range(1,lag+1)]

    for i,k in enumerate(colunas):
        new_df[k] = new_df["dados"].shift(+i+1)
    new_df=new_df.dropna()
    return new_df

#Função que cria o dataset necessário para as timeseries. As time series são todas unidas numa só coluna e os países respetivos
# a essa série temporal recebem um 1. Também concatena os lags no início da df.

def time_series(df, lag):
    binario = LabelBinarizer().fit(df.columns)
    df_final= pd.DataFrame()
    for pais in df.columns:
        pais_bin=pd.DataFrame(data=list(binario.transform([pais]))*len(df[pais]),
                             columns=binario.classes_,
                             index=df.index)
        #Agora juntamos os lags e os dados respetivos
        df_mid=pd.DataFrame()
        df_mid=pd.concat((df_mid,lags(df, lag, pais),pais_bin),axis=1).dropna()

        #Finalmente, concatenamos tudo numa só DataFrame
        df_final = pd.concat((df_final, df_mid), sort=True)
        df_fina=df_final.dropna()
    return df_final

# Função para criar um dataframe X para um dado dataset, pais e ano, este dataframe depois eh utilizado para fazer predict
def createX(dataset : pd.DataFrame, country : str, year : int, lag) -> pd.DataFrame:
    previousYearCountryData = dataset.loc[(dataset[country] == 1) & (dataset.index.get_level_values(0) == str(year - 1))]
    newX = previousYearCountryData.copy()
    newX.index = [str(year)]
    for i in range(lag,2,-1):
        newX.at[str(year), str(i)+"lag"] = newX.at[str(year),str(i-1)+"lag"]
    newX.at[str(year), "1lag"] = newX.at[str(year),"dados"]
    newX.drop(["dados"], axis=1, inplace=True)
    return newX

# Função para prever dados novos para um dado dataset ano e pais a partir do modelo treinado dado
def predictAndCalculateDelta(model, dataset : pd.DataFrame, country : str, year : int, lag):
    previousYearCountryData = dataset.loc[(dataset[country] == 1) & (dataset.index.get_level_values(0) == str(year - 1))]
    newX = createX(dataset, country, year, lag)
    x = model.predict(newX)
    x_previous = previousYearCountryData.at[str(year-1),"dados"]
    delta = x - x_previous
    return x[0], delta[0]

# Vai buscar o valor para um dado ano e pais e calcula o delta
def determineRealResults(dataset : pd.DataFrame, country : str, year : int):
    real_data = dataset.at[str(year), country]
    previous_year_data = dataset.at[str(year-1), country]
    delta = real_data - previous_year_data
    return real_data, delta

# Erro relativo e absoluto
def calculateErrors(real, pred):
    abs_error = abs(pred - real)
    relative_error = abs_error / abs(real)
    return abs_error, (relative_error*100)

# Criar dataframe com os resultados finais
def DFCreate(PrevisionData, year):
    # Criar dataframe
    df = pd.DataFrame(PrevisionData, columns=['Country', str(year) + ' Real Value', str(year) + ' Prevision','Relative Error %',str(year) + ' Real Delta',str(year) + ' Delta Prevision','Relative Error %'])
    # Converter valores em notacao legivel
    df = df.loc[:, df.columns == 'Country'].join(df.loc[:, df.columns != 'Country'].applymap(lambda x: '{:,.3f}'.format(x)))
    return df

Importação dos datasets

In [177]:
# Dados 1960-2016
fertility = pd.read_csv("fertility_rate.csv")
population= pd.read_csv("country_population.csv")
expectancy = pd.read_csv("life_expectancy.csv")

fertility=drop_nan(fertility,4)
population=drop_nan(population,4)
expectancy=drop_nan(expectancy,4)

# Dados 1960-2020
fertility_2020 = pd.read_csv("fertility_2020.csv")
population_2020 = pd.read_csv("population_2020.csv")
expectancy_2020 = pd.read_csv("life_2020.csv")

#Criar os data frames
fert=indexing(fertility, 1960)
pop=indexing(population, 1960)
exp=indexing(expectancy, 1960)

fert_2020 = indexing(fertility_2020, 1960)
pop_2020 = indexing(population_2020, 1960)
exp_2020 = indexing(expectancy_2020, 1960)

Vamos continuar a transformar o nosso dataset, de forma a que fiquem todas as séries temporais numa só coluna, a referência respetiva de que país se refere a série temporal e os respetivos lags para cada valor. 

In [178]:
#Versão final dos datasets
pop_fin=time_series(pop,4)
fert_fin=time_series(fert,4)
exp_fin=time_series(exp,4)

# Divisao em X e y
y_pop = pop_fin["dados"]
X_pop = pop_fin[list(pop_fin.columns)[:-1]]

y_fert = fert_fin["dados"]
X_fert = fert_fin[list(fert_fin.columns)[:-1]]

y_exp = exp_fin["dados"]
X_exp = exp_fin[list(exp_fin.columns)[:-1]]

Com isto feito, podemos passar ao treino dos modelos

## Escolha dos parametros e treino de dos modelos

### Populacao

Treino de 2 modelos, LinearRegression e DecisionTreeRegressor

In [179]:
# LinearRegression
params_linear = [
    {"alpha" : [x*0.1 for x in range(1,10)]}]

grid_search_lasso_pop = GridSearchCV(
    Lasso(), params_linear, scoring="neg_mean_squared_error", cv=10, n_jobs=-1)

grid_search_lasso_pop.fit(X_pop, y_pop)
print(grid_search_lasso_pop.best_params_)
print("Melhores Parâmetros:", grid_search_lasso_pop.best_params_,"\n")
preds= grid_search_lasso_pop.predict(X_pop)
printRegStatistics(y_pop, preds)

{'alpha': 0.9}
Melhores Parâmetros: {'alpha': 0.9} 

The RVE is:  0.9999945436199671
The rmse is:  1619501.3254123626
The Correlation Score is is: 1.0000 (p-value=0.000000e+00)

The Maximum Error is is:  11948798.81523943
The Mean Absolute Error is:  530556.1048894892


In [180]:
# DecisionTreeRegressor
params =[{"max_depth" : [4,8,12,15,20]}]

grid_search_dtr_pop = GridSearchCV(
    DecisionTreeRegressor(), params, scoring="neg_mean_squared_error", cv=10, n_jobs=-1)


grid_search_dtr_pop.fit(X_pop, y_pop)
print(grid_search_dtr_pop.best_params_)
print("Melhores Parâmetros:", grid_search_dtr_pop.best_params_,"\n")
preds= grid_search_dtr_pop.predict(X_pop)
printRegStatistics(y_pop, preds)

{'max_depth': 20}
Melhores Parâmetros: {'max_depth': 20} 

The RVE is:  0.9999999999876886
The rmse is:  2432.6678245585636
The Correlation Score is is: 1.0000 (p-value=0.000000e+00)

The Maximum Error is is:  60189.0
The Mean Absolute Error is:  544.496726629835


In [181]:
# Melhor modelo eh o DecisionTreeRegressor
pop_model = grid_search_dtr_pop

### Fertility

Treino de 2 modelos, LinearRegression e DecisionTreeRegressor

In [182]:
# LinearRegression
params_linear = [
    {"alpha" : [x*0.1 for x in range(1,10)]}]

grid_search_lasso_fert = GridSearchCV(
    Lasso(), params_linear, scoring="neg_mean_squared_error", cv=10, n_jobs=-1)

grid_search_lasso_fert.fit(X_fert, y_fert)
print(grid_search_lasso_fert.best_params_)
print("Melhores Parâmetros:", grid_search_lasso_fert.best_params_,"\n")
preds= grid_search_lasso_fert.predict(X_fert)
printRegStatistics(y_fert, preds)

{'alpha': 0.1}
Melhores Parâmetros: {'alpha': 0.1} 

The RVE is:  0.9979575888734916
The rmse is:  0.08871661995659094
The Correlation Score is is: 0.9993 (p-value=0.000000e+00)

The Maximum Error is is:  1.7454652135665294
The Mean Absolute Error is:  0.06636704938394361


In [183]:
# DecisionTreeRegressor
params =[{"max_depth" : [4,8,12,15,20]}]

grid_search_dtr_fert= GridSearchCV(
    DecisionTreeRegressor(), params, scoring="neg_mean_squared_error", cv=10, n_jobs=-1)


grid_search_dtr_fert.fit(X_fert, y_fert)
print(grid_search_dtr_fert.best_params_)
print("Melhores Parâmetros:", grid_search_dtr_fert.best_params_,"\n")
preds= grid_search_dtr_fert.predict(X_fert)
printRegStatistics(y_fert, preds)

{'max_depth': 12}
Melhores Parâmetros: {'max_depth': 12} 

The RVE is:  0.9998439148920937
The rmse is:  0.024525289228626297
The Correlation Score is is: 0.9999 (p-value=0.000000e+00)

The Maximum Error is is:  0.23823142179259227
The Mean Absolute Error is:  0.015067240931490858


In [184]:
# Melhor modelo eh o DecisionTreeRegressor
fertility_model = grid_search_dtr_fert

### Life Expectancy

Treino de 2 modelos, LinearRegression e DecisionTreeRegressor

In [185]:
# LinearRegression
params_linear = [
    {"alpha" : [x*0.01 for x in range(1,20)]}]

grid_search_lasso_exp = GridSearchCV(
    Lasso(), params_linear, scoring="neg_mean_squared_error", cv=10, n_jobs=-1)

grid_search_lasso_exp.fit(X_exp, y_exp)
print(grid_search_lasso_exp.best_params_)
print("Melhores Parâmetros:", grid_search_lasso_exp.best_params_,"\n")
preds= grid_search_lasso_exp.predict(X_exp)
printRegStatistics(y_exp, preds)

{'alpha': 0.01}
Melhores Parâmetros: {'alpha': 0.01} 

The RVE is:  0.9995256429514703
The rmse is:  0.23546571634644062
The Correlation Score is is: 0.9998 (p-value=0.000000e+00)

The Maximum Error is is:  3.4670980408972376
The Mean Absolute Error is:  0.11483406249588855


In [186]:
# DecisionTreeRegressor
params =[{"max_depth" : [4,8,12,15,20,25,30]}]

grid_search_dtr_exp = GridSearchCV(
    DecisionTreeRegressor(), params, scoring="neg_mean_squared_error", cv=10, n_jobs=-1)


grid_search_dtr_exp.fit(X_exp, y_exp)
print(grid_search_dtr_exp.best_params_)
print("Melhores Parâmetros:", grid_search_dtr_exp.best_params_,"\n")
preds= grid_search_dtr_exp.predict(X_exp)
printRegStatistics(y_exp, preds)

{'max_depth': 30}
Melhores Parâmetros: {'max_depth': 30} 

The RVE is:  0.9999999991384322
The rmse is:  0.00031733629849540774
The Correlation Score is is: 1.0000 (p-value=0.000000e+00)

The Maximum Error is is:  0.019190433333335477
The Mean Absolute Error is:  1.1331443352986805e-05


In [187]:
# Melhor modelo eh o DecisionTreeRegressor
expectancy_model = grid_search_dtr_exp

## Previsao de Valores de 2017

Escolher a sample de 10 países aleatórios

In [188]:
sample = list(fertility["Country Name"].sample(n=10, random_state=483))
print(sample)

['Lesotho', 'North America', 'New Zealand', 'Singapore', 'Europe & Central Asia', 'Qatar', 'Uzbekistan', 'Estonia', 'Angola', 'Japan']


### Populacao

In [189]:
PrevisionDataPop = []

for country in sample:
    x_prevision, delta_prevision = predictAndCalculateDelta(pop_model, pop_fin, country, 2017, lag=4)
    x_real, delta_real = determineRealResults(pop_2020, country, 2017)
    _, x_rel_error = calculateErrors(x_real, x_prevision)
    _, delta_rel_error = calculateErrors(delta_real, delta_prevision)
    data_entry =  (country, x_real, x_prevision, x_rel_error, delta_real, delta_prevision, delta_rel_error)
    PrevisionDataPop.append(data_entry)

# Criar dataframe
PopDF = DFCreate(PrevisionDataPop, 2017)
PopDF

,Country,2017 Real Value,2017 Prevision,Relative Error %,2017 Real Delta,2017 Delta Prevision,Relative Error %
0,Lesotho,"2,170,617.000","2,235,327.000",2.981,"26,745.000","31,506.000",17.801
1,North America,"361,731,237.000","366,468,511.000",1.310,"2,485,441.000","7,011,018.000",182.083
2,New Zealand,"4,813,600.000","4,795,832.000",0.369,"99,500.000","102,632.000",3.148
3,Singapore,"5,612,253.000","5,710,072.167",1.743,"4,970.000","102,789.167","1,968.192"
4,Europe & Central Asia,"915,855,416.000","921,702,426.000",0.638,"3,480,711.000","9,855,148.000",183.136
5,Qatar,"2,711,755.000","2,667,470.000",1.633,"116,589.000","97,666.000",16.231
6,Uzbekistan,"32,388,600.000","32,955,496.000",1.750,"540,700.000","1,107,596.000",104.845
7,Estonia,"1,317,384.000","1,315,790.000",0.121,"1,594.000",0.000,100.000
8,Angola,"30,208,628.000","29,550,662.000",2.178,"1,053,882.000","737,199.000",30.049
9,Japan,"126,972,000.000","126,994,511.000",0.018,"-104,000.000",0.000,100.000


### Fertility Rate

In [190]:
PrevisionDataFert = []

for country in sample:
    x_prevision, delta_prevision = predictAndCalculateDelta(fertility_model, fert_fin, country, 2017, lag=4)
    x_real, delta_real = determineRealResults(fert_2020, country, 2017)
    _, x_rel_error = calculateErrors(x_real, x_prevision)
    _, delta_rel_error = calculateErrors(delta_real, delta_prevision)
    data_entry =  (country, x_real, x_prevision, x_rel_error, delta_real, delta_prevision, delta_rel_error)
    PrevisionDataFert.append(data_entry)

# Criar dataframe
FerDF = DFCreate(PrevisionDataFert, 2017)
FerDF

,Country,2017 Real Value,2017 Prevision,Relative Error %,2017 Real Delta,2017 Delta Prevision,Relative Error %
0,Lesotho,3.187,3.063,3.891,-0.052,-0.031,40.358
1,North America,1.743,1.745,0.112,-0.055,-0.035,35.883
2,New Zealand,1.810,1.829,1.047,-0.060,-0.041,31.573
3,Singapore,1.160,1.227,5.771,-0.040,0.027,167.353
4,Europe & Central Asia,1.735,1.763,1.621,-0.047,0.013,127.569
5,Qatar,1.839,1.890,2.785,0.012,-0.017,239.800
6,Uzbekistan,2.419,2.436,0.723,-0.036,-0.019,48.579
7,Estonia,1.590,1.566,1.523,-0.010,-0.014,42.228
8,Angola,5.600,5.617,0.305,-0.086,-0.077,10.589
9,Japan,1.430,1.451,1.463,-0.010,0.011,209.193


### Life Expectancy

In [191]:
PrevisionDataExp = []

for country in sample:
    x_prevision, delta_prevision = predictAndCalculateDelta(expectancy_model, exp_fin, country, 2017, lag=4)
    x_real, delta_real = determineRealResults(exp_2020, country, 2017)
    _, x_rel_error = calculateErrors(x_real, x_prevision)
    _, delta_rel_error = calculateErrors(delta_real, delta_prevision)
    data_entry =  (country, x_real, x_prevision, x_rel_error, delta_real, delta_prevision, delta_rel_error)
    PrevisionDataExp.append(data_entry)

# Criar dataframe
ExpDF = DFCreate(PrevisionDataExp, 2017)
ExpDF

,Country,2017 Real Value,2017 Prevision,Relative Error %,2017 Real Delta,2017 Delta Prevision,Relative Error %
0,Lesotho,53.064,54.709,3.100,0.800,0.535,33.125
1,North America,78.879,79.154,0.349,0.002,0.099,"5,539.643"
2,New Zealand,81.659,81.612,0.056,0.046,0.000,100.000
3,Singapore,83.095,82.843,0.304,0.249,0.048,80.882
4,Europe & Central Asia,77.779,77.741,0.048,0.213,0.459,115.178
5,Qatar,80.717,78.430,2.833,0.284,0.246,13.380
6,Uzbekistan,71.010,71.459,0.632,0.255,0.145,43.319
7,Estonia,78.093,77.678,0.531,0.451,-0.059,112.973
8,Angola,61.680,61.899,0.355,0.588,0.352,40.136
9,Japan,84.100,84.278,0.212,0.115,0.293,155.202
